# Métricas de concordância

Fontes importantes:
- Doc CSV: https://docs.python.org/3/library/csv.html
- Doc XML Element Tree: https://docs.python.org/3/library/xml.etree.elementtree.html

Esse script serve para salvar anotações em um arquivo CSV. Seu intuito é gerar uma representação de dados adequada à implementação de métricas de concordância.

### Append de arquivos xml e instanciação de lista de roots

In [1]:
import os

xmls = os.listdir('./xmls')

In [2]:
xmls

['17_133.28.6.2013_v1.xml',
 '11_243.20.11.2013_v1.xml',
 '19_152.28.7.2014_v1.xml',
 '20_147.21.7.2014_v1.xml',
 '16_167.14.8.2013_v1.xml',
 '8_200.26.9.2013_v1.xml',
 '10_193.17.9.2013_v1.xml',
 '6_191.13.9.2013_v1.xml',
 '12_244.21.11.2013_v1.xml',
 '4_2.3.1.2013_v1.xml',
 '5_56.19.3.2013_v1.xml',
 '15_166.13.8.2013_v1.xml',
 '7_188.10.9.2013_v1.xml',
 '14_204.1.10.2013_v1.xml',
 '18_119.12.6.2013_v1.xml',
 '9_202.27.9.2013_v1.xml',
 '13_212.10.10.2013_v1.xml']

In [3]:
import xml.etree.ElementTree as ET

# lista de roots para iteração posterior
roots = []

# populando lista de roots e iterando na lista xmls
for xml in xmls:
    xml = './xmls/' + xml
    tree = ET.parse(xml)
    root = tree.getroot()
    roots.append(root)

### Definição das colunas do CSV

O CSV foi montado com informações provientes do NidoTat. A ideia foi reunir todos os campos que podem vir a ser úteis na alimentação de métricas de concordância.

As colunas selecionadas foram:
- Id do DODF
- Tipo da relação
- Id da relação
- Anotador da relação
- Tipo da entidade
- Id da entidade
- Anotador da entidade
- Offset da entidade
- Length da entidade
- Texto da entidade

In [4]:
import csv

# abre csv para escrita de headers
with open('agreement.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(['id_dodf', 'tipo_rel', 'id_rel', 'anotador_rel', 'tipo_ent', 'id_ent', 'anotador_ent', 'offset', 'length', 'texto'])

### XML -> CSV

In [5]:
from tqdm.notebook import tqdm

# abre csv para escrita em modo append
with open('agreement.csv', 'a') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    
    # itera na lista de roots
    for root in tqdm(roots):
        # coleta id do dodf
        id_dodf = root.find("./document/id")
        id_dodf_text = id_dodf.text
        # cria lista de ids de relações
        ids_rel = []
        for rel in root.findall("./document/passage/relation"):
            id_rel = rel.get('id')
            ids_rel.append(id_rel)
            # coleta tipo e anotador da relação
            for info in rel.findall('infon'):
                if info.get('key') == 'type':
                    tipoRel = info.text
                    #print(tipoRel)
                elif info.get('key') == 'annotator':
                    annotatorRel = info.text
                    #print(annotatorRel)
            # cria lista de ids de anotações
            ids_anno = []
            for info in rel.findall('node'):
                id_anno = info.get('refid')
                ids_anno.append(id_anno)
            #print(ids_anno)
            # loop na lista de ids
            for id_anno in ids_anno:
                # encontra e itera sobre todos os elementos annotation do xml
                for anno in root.findall("./document/passage/annotation"):
                    # para cada anotação definida por um id, coleta o tipo, anotador, offset, length e texto
                    if anno.get('id') == id_anno:
                        # encontra tipo e anotador
                        for info in anno.findall('infon'):
                            if info.get('key') == 'type':
                                tipoAnno = info.text
                                #print(tipoAnno)
                            elif info.get('key') == 'annotator':
                                annotatorAnno = info.text
                                #print(annotatorAnno)
                        # encontra offset e length
                        for info in anno.findall('location'):
                            offset = info.get('offset')
                            #print(offset)
                            length = info.get('length')
                            #print(length)
                        # encontra texto
                        for info in anno.findall('text'):
                            texto = info.text
                            #print(texto)
                        # escreve linha no csv
                        writer.writerow([id_dodf_text, tipoRel, id_rel, annotatorRel, tipoAnno, id_anno, annotatorAnno, offset, length, texto])

  0%|          | 0/17 [00:00<?, ?it/s]

In [6]:
import pandas as pd

df = pd.read_csv('agreement.csv')
df

,id_dodf,tipo_rel,id_rel,anotador_rel,tipo_ent,id_ent,anotador_ent,offset,length,texto
0,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,Ato_Exoneracao_Comissionado,1178,pedro_henrique,205008,326,EXONERAR SEBASTIAO FRANCISCO DE QUEIROZ do Car...
1,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,nome,1179,DODFMiner,205016,31,SEBASTIAO FRANCISCO DE QUEIROZ
2,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,simbolo,1181,DODFMiner,205078,4,DFA-
3,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,simbolo,1180,pedro_henrique,205078,7,DFA-\n12
4,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,cargo_comissionado,1182,DODFMiner,205089,9,Assessor
...,...,...,...,...,...,...,...,...,...,...
19487,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,matricula_substituido,8685,vinicius_borges,644324,9,174.833-5
19488,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,data_inicial,8686,vinicius_borges,644348,2,09
19489,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,data_final,8687,vinicius_borges,644353,10,11/10/2013
19490,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,motivo,8688,vinicius_borges,644380,32,ferias regulamentares da titular


In [136]:
df.tipo_rel.value_counts()

Ato_Exoneracao_Comissionado       3891
Ato_Nomeacao_Comissionado         3851
Ato_Tornado_Sem_Efeito_Exo_Nom    2815
Ato_Retificacao_Comissionado      2691
Ato_Retificacao_Efetivo           1895
Ato_Substituicao                  1793
Ato_Exoneracao_Efetivo            1555
Ato_Cessao                         370
Ato_Nomeacao_Efetivo               197
Ato_Abono_Permanencia              175
Ato_Tornado_Sem_Efeito_Apo         170
Ato_Reversao                        89
Name: tipo_rel, dtype: int64

In [137]:
df.tipo_ent.value_counts()

nome                          2022
orgao                         1600
hierarquia_lotacao            1512
cargo_comissionado            1411
simbolo                       1358
                              ... 
matricula_siape                  3
data_dodf_edital_normativo       3
data                             3
tipo_edicao                      2
especialidade                    1
Name: tipo_ent, Length: 69, dtype: int64

In [138]:
df.anotador_rel.value_counts()

jose_reinaldo       3851
renato_nobre        3667
livia_fonseca       2564
matheus_stauffer    2413
khalil_carsten      2189
leonardo_maffei     1549
tatiana_franco      1443
pedro_henrique      1432
thiago_faleiros      384
Name: anotador_rel, dtype: int64

In [139]:
df.anotador_ent.value_counts()

DODFMiner           4798
lindeberg           2878
tatiana_franco      2428
matheus_stauffer    2083
vinicius_borges     2057
patricia_medyna     1219
livia_fonseca       1118
pedro_henrique      1066
teo_decampos         753
leonardo_maffei      716
alvesisaque          141
jose_reinaldo        129
fredguth              60
gerente_matheus       28
khalil_carsten        13
thiago_faleiros        4
renato_nobre           1
Name: anotador_ent, dtype: int64

## Criação de estruturas auxiliares em memória

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('agreement.csv')
df

,id_dodf,tipo_rel,id_rel,anotador_rel,tipo_ent,id_ent,anotador_ent,offset,length,texto
0,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,Ato_Exoneracao_Comissionado,1178,pedro_henrique,205008,326,EXONERAR SEBASTIAO FRANCISCO DE QUEIROZ do Car...
1,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,nome,1179,DODFMiner,205016,31,SEBASTIAO FRANCISCO DE QUEIROZ
2,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,simbolo,1181,DODFMiner,205078,4,DFA-
3,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,simbolo,1180,pedro_henrique,205078,7,DFA-\n12
4,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,cargo_comissionado,1182,DODFMiner,205089,9,Assessor
...,...,...,...,...,...,...,...,...,...,...
19487,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,matricula_substituido,8685,vinicius_borges,644324,9,174.833-5
19488,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,data_inicial,8686,vinicius_borges,644348,2,09
19489,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,data_final,8687,vinicius_borges,644353,10,11/10/2013
19490,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,motivo,8688,vinicius_borges,644380,32,ferias regulamentares da titular


Para correlação offset/length/user, parece mais fácil extrair apenas a listagem em ordem e depois lidar com duplicatas.

In [2]:
offsets = df.offset.to_list()
offsets

[205008,
 205016,
 205078,
 205078,
 205089,
 205103,
 205103,
 205286,
 205301,
 205008,
 205016,
 205078,
 205078,
 205089,
 205103,
 205103,
 205286,
 205301,
 205008,
 205016,
 205078,
 205078,
 205089,
 205103,
 205103,
 205286,
 205301,
 208583,
 208720,
 208786,
 208573,
 208629,
 208677,
 208732,
 208753,
 205008,
 205016,
 205078,
 205078,
 205089,
 205286,
 205103,
 205840,
 205855,
 205560,
 205570,
 205611,
 205667,
 205677,
 205693,
 209173,
 209311,
 209377,
 209344,
 209163,
 209217,
 209219,
 209230,
 209268,
 209322,
 205397,
 205511,
 205526,
 205336,
 205344,
 205387,
 205411,
 206086,
 206101,
 205962,
 205895,
 205889,
 205974,
 205987,
 205987,
 206214,
 206135,
 206141,
 206225,
 206240,
 206240,
 206372,
 206387,
 206808,
 206715,
 206721,
 206819,
 206831,
 206831,
 206935,
 207088,
 206985,
 206991,
 207077,
 207101,
 207171,
 207540,
 207481,
 207490,
 207500,
 207544,
 207594,
 207722,
 208440,
 208372,
 208539,
 208317,
 208376,
 208308,
 208326,
 208450,
 

In [3]:
lengths = df.length.to_list()
lengths

[326,
 31,
 4,
 7,
 9,
 193,
 178,
 47,
 32,
 326,
 31,
 4,
 7,
 9,
 193,
 178,
 47,
 32,
 326,
 31,
 4,
 7,
 9,
 193,
 178,
 47,
 32,
 40,
 6,
 32,
 246,
 32,
 11,
 16,
 28,
 326,
 31,
 7,
 4,
 9,
 47,
 178,
 47,
 32,
 328,
 40,
 24,
 6,
 11,
 142,
 40,
 6,
 32,
 28,
 247,
 12,
 22,
 22,
 11,
 17,
 9,
 47,
 32,
 223,
 12,
 6,
 94,
 47,
 32,
 6,
 24,
 245,
 8,
 109,
 94,
 6,
 286,
 29,
 10,
 127,
 142,
 48,
 33,
 6,
 269,
 34,
 7,
 99,
 114,
 48,
 8,
 238,
 42,
 6,
 65,
 51,
 3,
 440,
 9,
 28,
 12,
 123,
 33,
 6,
 3,
 32,
 8,
 11,
 264,
 33,
 9,
 69,
 33,
 18,
 19,
 11,
 342,
 11,
 21,
 6,
 17,
 32,
 12,
 11,
 218,
 20,
 34,
 6,
 32,
 16,
 12,
 217,
 31,
 8,
 29,
 32,
 16,
 397,
 12,
 21,
 8,
 82,
 40,
 256,
 40,
 11,
 6,
 17,
 33,
 32,
 40,
 32,
 385,
 44,
 8,
 34,
 11,
 18,
 19,
 8,
 52,
 7,
 18,
 2,
 18,
 27,
 6,
 43,
 2,
 364,
 8,
 52,
 7,
 18,
 2,
 18,
 27,
 6,
 43,
 2,
 364,
 8,
 52,
 7,
 18,
 2,
 18,
 27,
 6,
 43,
 2,
 364,
 6,
 8,
 332,
 27,
 82,
 154,
 8,
 52,
 364,
 7,
 18,
 

In [4]:
texts = df.texto.to_list()
texts

['EXONERAR SEBASTIAO FRANCISCO DE QUEIROZ do Cargo em Comissao, Simbolo DFA-\n12, de Assessor, da Gerencia Regional do Setor Complementar de Industria e Abastecimento, da \nAdministracao Regional do Setor Complementar de Industria e Abastecimento, da Coordenadoria \ndas Cidades, da Casa Civil, da Governadoria do Distrito Federal.',
 ' SEBASTIAO FRANCISCO DE QUEIROZ',
 'DFA-',
 'DFA-\n12',
 ' Assessor',
 'Gerencia Regional do Setor Complementar de Industria e Abastecimento, da \nAdministracao Regional do Setor Complementar de Industria e Abastecimento, da Coordenadoria \ndas Cidades, da Casa Civil',
 'Gerencia Regional do Setor Complementar de Industria e Abastecimento, da \nAdministracao Regional do Setor Complementar de Industria e Abastecimento, da Coordenadoria \ndas Cidades',
 'Casa Civil, da Governadoria do Distrito Federal',
 'Governadoria do Distrito Federal',
 'EXONERAR SEBASTIAO FRANCISCO DE QUEIROZ do Cargo em Comissao, Simbolo DFA-\n12, de Assessor, da Gerencia Regional do S

In [5]:
anotators = df.anotador_ent.to_list()
anotators

['pedro_henrique',
 'DODFMiner',
 'DODFMiner',
 'pedro_henrique',
 'DODFMiner',
 'DODFMiner',
 'pedro_henrique',
 'pedro_henrique',
 'DODFMiner',
 'pedro_henrique',
 'DODFMiner',
 'DODFMiner',
 'pedro_henrique',
 'DODFMiner',
 'DODFMiner',
 'pedro_henrique',
 'pedro_henrique',
 'DODFMiner',
 'pedro_henrique',
 'DODFMiner',
 'DODFMiner',
 'pedro_henrique',
 'DODFMiner',
 'DODFMiner',
 'pedro_henrique',
 'pedro_henrique',
 'DODFMiner',
 'pedro_henrique',
 'DODFMiner',
 'pedro_henrique',
 'pedro_henrique',
 'DODFMiner',
 'pedro_henrique',
 'pedro_henrique',
 'pedro_henrique',
 'pedro_henrique',
 'DODFMiner',
 'pedro_henrique',
 'DODFMiner',
 'DODFMiner',
 'pedro_henrique',
 'matheus_stauffer',
 'pedro_henrique',
 'DODFMiner',
 'pedro_henrique',
 'pedro_henrique',
 'DODFMiner',
 'DODFMiner',
 'DODFMiner',
 'matheus_stauffer',
 'pedro_henrique',
 'pedro_henrique',
 'pedro_henrique',
 'pedro_henrique',
 'pedro_henrique',
 'pedro_henrique',
 'DODFMiner',
 'pedro_henrique',
 'pedro_henrique',


In [6]:
off_sem_duplicata = np.unique(df.offset.to_numpy())

In [7]:
off_sem_duplicata

array([   304,    305,    312, ..., 644348, 644353, 644380])

In [8]:
map_duplicata = {}
for i in off_sem_duplicata:
    map_duplicata[i] = []
for idx, offset in enumerate(offsets):
    map_duplicata[offset].append(idx)

`map_duplicata` é um dicionário cujo objetivo é dispor em uma estrutura de dados todas as ocorrências de um mesmo offset. Para tanto, mapeamos os offsets e linhas do dataframe original. As chaves são todos os offsets únicos, e os valores são listas contendo as linhas onde aquele offset ocorre. Com esse mapa, poderemos iterar entre os valores e adotar as diferentes métricas que quisermos.

In [9]:
map_duplicata

{304: [15160],
 305: [1494],
 312: [9320, 9443, 9566, 9677],
 314: [15161],
 315: [1500],
 323: [15162],
 338: [9321, 9444, 9567, 9678],
 356: [1495],
 359: [12575],
 363: [9322, 9445, 9568, 9679, 15163],
 364: [9323, 9446, 9569, 9680],
 369: [12576],
 378: [12577],
 407: [9324, 9447, 9570, 9681],
 414: [15164],
 424: [15165],
 430: [1496],
 435: [9325, 9448, 9571, 9682],
 439: [12578],
 440: [1497],
 443: [15166],
 445: [9326, 9449, 9572, 9683],
 446: [9327, 9450, 9573, 9684],
 449: [12579],
 451: [1498],
 465: [12580],
 466: [1499],
 468: [9328, 9451, 9574, 9685],
 502: [1446],
 508: [1447],
 522: [9329, 9452, 9575, 9686],
 556: [9389, 9512, 9746, 9776],
 566: [1448],
 577: [9390, 9513, 9747, 9777, 15167],
 588: [9391, 9514, 9748, 9778],
 594: [1449],
 612: [15152],
 622: [15153],
 632: [9392, 9515, 9749, 9779],
 641: [9393, 9516, 9750, 9780],
 663: [15154],
 664: [15155],
 671: [9394, 9517, 9751, 9781],
 681: [1450],
 692: [9395, 9518, 9752, 9782],
 694: [12581],
 703: [15156],
 715

In [10]:
len(map_duplicata)

15566

In [11]:
map_duplicata[312]

[9320, 9443, 9566, 9677]

Para facilitar a consulta aos dados, vamos criar um dicionário `data_dict`, cujas chaves são linhas do dataframe e os valores são tuplas contendo `(offset, length, anotador_ent, texto)` da respectiva linha.

In [12]:
data_dict = {}
for idx, _ in enumerate(offsets):
    data_dict[idx] = (offsets[idx], lengths[idx], anotators[idx], texts[idx])

In [13]:
data_dict

{0: (205008,
  326,
  'pedro_henrique',
  'EXONERAR SEBASTIAO FRANCISCO DE QUEIROZ do Cargo em Comissao, Simbolo DFA-\n12, de Assessor, da Gerencia Regional do Setor Complementar de Industria e Abastecimento, da \nAdministracao Regional do Setor Complementar de Industria e Abastecimento, da Coordenadoria \ndas Cidades, da Casa Civil, da Governadoria do Distrito Federal.'),
 1: (205016, 31, 'DODFMiner', ' SEBASTIAO FRANCISCO DE QUEIROZ'),
 2: (205078, 4, 'DODFMiner', 'DFA-'),
 3: (205078, 7, 'pedro_henrique', 'DFA-\n12'),
 4: (205089, 9, 'DODFMiner', ' Assessor'),
 5: (205103,
  193,
  'DODFMiner',
  'Gerencia Regional do Setor Complementar de Industria e Abastecimento, da \nAdministracao Regional do Setor Complementar de Industria e Abastecimento, da Coordenadoria \ndas Cidades, da Casa Civil'),
 6: (205103,
  178,
  'pedro_henrique',
  'Gerencia Regional do Setor Complementar de Industria e Abastecimento, da \nAdministracao Regional do Setor Complementar de Industria e Abastecimento, 

In [14]:
len(data_dict)

19492

### Summary: estruturas auxiliares
A ideia é iterar em `map_duplicata`, obtendo a cada iteração listas de linhas do dataframe, e então usar `data_dict`, cujas chaves são as linhas do dataframe, para realizar o processamento necessário.

## Jaccard

O coeficiente de Jaccard foi originalmente criado para imagens, mas podemos adaptar para dados textuais.

<img src="jaccard.png" alt="Drawing" style="width: 600px;"/>

<img src="jaccard2.png" alt="Drawing" style="width: 600px;"/>

Fonte das imagens: https://medium.com/data-science-bootcamp/understand-jaccard-index-jaccard-similarity-in-minutes-25a703fbf9d7

Na prática, nossa adaptação para textos vai assumir que a união entre as strings A e B é a string de maior tamanho. Como estamos comparando - em tese - strings parecidas, pensando a nível de caractere faz sentido pensar nessa união como um overlap de textos parecidos. Essa premissa já tras consigo um certo bias, mas acho que é uma assumption aceitável. Se a união é o overlap, a interseção é justamente esse overlap menos o que A e B têm de diferente. Basta encontrar a quantidade de caracteres que A e B diferem e tomar o módulo da diferença com a string de maior tamanho.

Em uma formulinha, ficaria assim:

$$\text{Jaccard}(A, B) = \frac{|{\text{tam_str} - \text{diferenca_char}}|}{\text{tam_str}}$$

onde _tam_str_ é o tamanho de uma das strings avaliadas - o módulo permite pegar qualquer uma - e _diferenca_char_ a diferença de caracteres entre A e B.

In [15]:
def jaccard_char(str1, str2):
    diferenca_char = 0
    #print(f'str1: {str1}')
    #print(f'str2: {str2}')
    # fazer assert fora: se ambas as strings possuem o mesmo valor de offset
    if len(str2) > len(str1):
        for idx, _ in enumerate(str2):
            try:
                if str2[idx] != str1[idx]:
                    diferenca_char += 1
            except:
                dif_reman = len(str2) - idx
                diferenca_char += dif_reman
                break
        agreement = abs(len(str2) - diferenca_char)/len(str2)
    else:
        for idx, _ in enumerate(str1):
            try:
                if str1[idx] != str2[idx]:
                    diferenca_char += 1
            except:
                dif_reman = len(str1) - idx
                diferenca_char += dif_reman
                break
        agreement = abs(len(str1) - diferenca_char)/len(str1)
    
    return agreement

Como exemplo de uso, vamos pegar essas duas anotações de símbolo:

In [16]:
data_dict[2][3], data_dict[3][3]

('DFA-', 'DFA-\n12')

In [17]:
agreement = jaccard_char(data_dict[2][3], data_dict[3][3])
agreement

0.5714285714285714

Ou, pensando em diferença:

In [18]:
dif = 1 - agreement
dif

0.4285714285714286

## Similaridade de cossenos

Outra abordagem possível para medir diferença entre textos é usar técnicas baseadas em representações vetoriais, como a similaridade de cossenos e distância euclidiana.

In [19]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

df = pd.read_csv('agreement.csv')
df

,id_dodf,tipo_rel,id_rel,anotador_rel,tipo_ent,id_ent,anotador_ent,offset,length,texto
0,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,Ato_Exoneracao_Comissionado,1178,pedro_henrique,205008,326,EXONERAR SEBASTIAO FRANCISCO DE QUEIROZ do Car...
1,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,nome,1179,DODFMiner,205016,31,SEBASTIAO FRANCISCO DE QUEIROZ
2,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,simbolo,1181,DODFMiner,205078,4,DFA-
3,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,simbolo,1180,pedro_henrique,205078,7,DFA-\n12
4,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,cargo_comissionado,1182,DODFMiner,205089,9,Assessor
...,...,...,...,...,...,...,...,...,...,...
19487,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,matricula_substituido,8685,vinicius_borges,644324,9,174.833-5
19488,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,data_inicial,8686,vinicius_borges,644348,2,09
19489,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,data_final,8687,vinicius_borges,644353,10,11/10/2013
19490,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,motivo,8688,vinicius_borges,644380,32,ferias regulamentares da titular


A ideia aqui é criar uma representação vetorial de nossos dados a partir do algoritmo word2vec.

Inicialmente, vamos criar duas listas: uma contendo todos os textos de anotações e outra contendo todos os termos, vulgo tokens, de cada texto de anotação. Essa última será uma lista de listas.

In [20]:
anotacoes = df['texto'].to_list()

In [21]:
len(anotacoes)

19492

In [22]:
anotacoes[0]

'EXONERAR SEBASTIAO FRANCISCO DE QUEIROZ do Cargo em Comissao, Simbolo DFA-\n12, de Assessor, da Gerencia Regional do Setor Complementar de Industria e Abastecimento, da \nAdministracao Regional do Setor Complementar de Industria e Abastecimento, da Coordenadoria \ndas Cidades, da Casa Civil, da Governadoria do Distrito Federal.'

Para a lista de tokens, vamos desprezar qualquer pré-processamento. Queremos obter uma representação dos dados o mais próximo possível da realidade, então usaremos apenas o método `split()` para separar os tokens de cada texto de anotação.

In [23]:
anotacoes[0].split()

['EXONERAR',
 'SEBASTIAO',
 'FRANCISCO',
 'DE',
 'QUEIROZ',
 'do',
 'Cargo',
 'em',
 'Comissao,',
 'Simbolo',
 'DFA-',
 '12,',
 'de',
 'Assessor,',
 'da',
 'Gerencia',
 'Regional',
 'do',
 'Setor',
 'Complementar',
 'de',
 'Industria',
 'e',
 'Abastecimento,',
 'da',
 'Administracao',
 'Regional',
 'do',
 'Setor',
 'Complementar',
 'de',
 'Industria',
 'e',
 'Abastecimento,',
 'da',
 'Coordenadoria',
 'das',
 'Cidades,',
 'da',
 'Casa',
 'Civil,',
 'da',
 'Governadoria',
 'do',
 'Distrito',
 'Federal.']

In [24]:
tokens = []

for texto in tqdm(anotacoes):
    token_list = texto.split()
    tokens.append(token_list)

  0%|          | 0/19492 [00:00<?, ?it/s]

In [25]:
len(tokens)

19492

Com a lista de tokens em mãos, o que falta agora é criar uma representação vetorial em si. Vamos usar a suíte de word2vec do gensim, que poupa muito trabalho. Basta passar como parâmetro a lista de tokens e a dimensão das embeddings - aqui vamos de 50. Os outros parâmetros são optativos; aqui usamos:
- window, que diz respeito à janela de contexto usada na geração dos vetores. Imagine o seguinte texto: 'embeddings são interessantes e úteis'. Se o valor escolhido é 3, o vetor gerado referente ao termos 'embeddings' será 'influenciado' pelos 3 termos seguintes, 'são', 'interessantes' e 'e';
- min_count, serve para ignorar os termos com frequência menor que o inteiro informado;
- workers, quantas worker threads serão usadas para treinar o modelo;
- negative, que diz o número de noise words para serem usadas no processo de negative sampling. Se o número setado for 0, negative sampling não é usado.

Mais detalhes constam em https://radimrehurek.com/gensim/models/word2vec.html.

In [26]:
from gensim.models import Word2Vec

In [27]:
model = Word2Vec(sentences=tokens, size=50, window=3, min_count=1, workers=4, negative=5)

In [28]:
vectors = model.wv

Temos vetores de como os tokens vieram nas anotações. Apenas para reforçar, usamos apenas split na criação da lista de termos propositalmente, para conseguir representar a realidade dos dados da maneira mais fiel possível. Temos vetores para strings como '11/10/2013', por exemplo. Esse caso que pode ocorrer em entidades como vigência ou data.

In [29]:
vectors['11/10/2013']

array([-0.05276367,  0.00490326,  0.0175669 ,  0.00038346,  0.02569353,
        0.00546703,  0.03936931,  0.011612  , -0.05560786, -0.01309053,
       -0.02091206,  0.05760368, -0.00359887, -0.05050317, -0.00946759,
       -0.00149195, -0.06660907, -0.01021693,  0.04082191,  0.0470452 ,
       -0.0105708 ,  0.00851833,  0.03843681, -0.00951724,  0.02718124,
       -0.01293199,  0.06220688,  0.01283669,  0.01143908,  0.0484998 ,
        0.02408719,  0.03073244, -0.03794201,  0.01854886, -0.03890786,
       -0.02040876, -0.00671752, -0.0195002 , -0.00274946, -0.00272406,
       -0.06749289,  0.01394136,  0.03999031, -0.04160165,  0.0025    ,
       -0.00472032,  0.02633209,  0.00337681,  0.01681907, -0.01635485],
      dtype=float32)

Criamos o modelo vetorial termo a termo. Agora, vamos obter vetores para os textos de anotação - que podem ter mais de um único token - somando a representação vetorial de cada termo no texto, de maneira a retornar um único vetor de 50 dimensões.

Para tanto, vamos criar uma função para facilitar nossa vida. A função `get_vector` recebe um texto como parâmetro e retorna uma representação vetorial correspondente.

In [30]:
def get_vector(text):
    token_list = text.split()
    vector = np.zeros(50)
    for token in token_list:
        vector += vectors[token]
    return vector

Para a primeira anotação do dataframe, temos o seguinte:

In [32]:
anotacoes[0]

'EXONERAR SEBASTIAO FRANCISCO DE QUEIROZ do Cargo em Comissao, Simbolo DFA-\n12, de Assessor, da Gerencia Regional do Setor Complementar de Industria e Abastecimento, da \nAdministracao Regional do Setor Complementar de Industria e Abastecimento, da Coordenadoria \ndas Cidades, da Casa Civil, da Governadoria do Distrito Federal.'

In [33]:
get_vector(anotacoes[0])

array([-37.34353318,  36.97200143, -13.00427554,  41.02821188,
        -7.81308767,  19.31773282,  40.3151405 ,  69.04490469,
         2.36683015, -58.06696894,   9.08575697,  31.42143092,
       -22.23337103, -15.97368591, -29.54459826,  62.94169183,
       -31.14048757, -26.74181773,  25.08960965,  16.856156  ,
       -16.99737589,  67.55657518,  72.98971202,  15.55767307,
        37.05903573,  24.90834895,  32.84149942, -22.03837222,
       -13.00686675,  81.64100557,  -0.3563193 ,  50.17507641,
       -25.52222665,   2.80164943, -23.46730085,  23.46665061,
       -49.33242552, -29.02328032,  -8.02186058, -19.66278761,
       -43.40193307,  39.63002846,  -4.60431459, -77.22844569,
        10.11525273,  68.57415006,  -6.99212504, -53.26973426,
       -20.22047881,  28.70517073])

Para auxiliar no mapeamento da referência da anotação de texto (linha do dataframe) e sua representação vetorial, vamos criar uma nova coluna index. A ideia é bem simples: criar uma lista de 19492 inteiros e agregar ao dataframe.

In [34]:
index = [x for x in range (19492)]

In [35]:
len(index)

19492

In [36]:
df['index'] = index

In [37]:
df

,id_dodf,tipo_rel,id_rel,anotador_rel,tipo_ent,id_ent,anotador_ent,offset,length,texto,index
0,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,Ato_Exoneracao_Comissionado,1178,pedro_henrique,205008,326,EXONERAR SEBASTIAO FRANCISCO DE QUEIROZ do Car...,0
1,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,nome,1179,DODFMiner,205016,31,SEBASTIAO FRANCISCO DE QUEIROZ,1
2,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,simbolo,1181,DODFMiner,205078,4,DFA-,2
3,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,simbolo,1180,pedro_henrique,205078,7,DFA-\n12,3
4,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,cargo_comissionado,1182,DODFMiner,205089,9,Assessor,4
...,...,...,...,...,...,...,...,...,...,...,...
19487,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,matricula_substituido,8685,vinicius_borges,644324,9,174.833-5,19487
19488,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,data_inicial,8686,vinicius_borges,644348,2,09,19488
19489,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,data_final,8687,vinicius_borges,644353,10,11/10/2013,19489
19490,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,motivo,8688,vinicius_borges,644380,32,ferias regulamentares da titular,19490


Já temos os elementos para processar em massa! Vamos criar um estrutura para mapear anotações e representações vetorias. Nesse sentido, temos o dicionário `map_texto_vetores`, quase autoexplicativo: mapeia textos e seus respectivos vetores. As listas `index` e `anotacoes` tem o mesmo tamanho. Iterando em `index`, teremos as mesmas referências para a lista `anotacoes`. Usamos a função `create_vector` feita mais cedo para retornar cada vetor.

_disclaimer_: Podia ter feito só um enumerate na lista `anotacoes` que aqui teria o mesmo resultado. De todo modo, criar uma coluna de index no dataframe pode ser útil em outros momentos do trabalho.

In [38]:
map_texto_vetores = {}

for i in tqdm(index):
    map_texto_vetores[i] = get_vector(anotacoes[i])

  0%|          | 0/19492 [00:00<?, ?it/s]

In [39]:
map_texto_vetores

{0: array([-37.34353318,  36.97200143, -13.00427554,  41.02821188,
         -7.81308767,  19.31773282,  40.3151405 ,  69.04490469,
          2.36683015, -58.06696894,   9.08575697,  31.42143092,
        -22.23337103, -15.97368591, -29.54459826,  62.94169183,
        -31.14048757, -26.74181773,  25.08960965,  16.856156  ,
        -16.99737589,  67.55657518,  72.98971202,  15.55767307,
         37.05903573,  24.90834895,  32.84149942, -22.03837222,
        -13.00686675,  81.64100557,  -0.3563193 ,  50.17507641,
        -25.52222665,   2.80164943, -23.46730085,  23.46665061,
        -49.33242552, -29.02328032,  -8.02186058, -19.66278761,
        -43.40193307,  39.63002846,  -4.60431459, -77.22844569,
         10.11525273,  68.57415006,  -6.99212504, -53.26973426,
        -20.22047881,  28.70517073]),
 1: array([-0.45434915,  6.28539455,  2.50236936,  1.20608336,  2.69836819,
         0.12740711,  5.48816419,  1.33500676, -1.98368689, -1.65106291,
        -2.15388043,  1.99887206,  0.80885

Mais cedo citamos similaridade de cossenos e distância euclidiana. No curso do Coursera, tivemos de fazer as duas implementações, que são bem simples. Segue na íntegra os enunciados e resultados.

The cosine similarity function is:

$$\cos (\theta)=\frac{\mathbf{A} \cdot \mathbf{B}}{\|\mathbf{A}\|\|\mathbf{B}\|}=\frac{\sum_{i=1}^{n} A_{i} B_{i}}{\sqrt{\sum_{i=1}^{n} A_{i}^{2}} \sqrt{\sum_{i=1}^{n} B_{i}^{2}}}\tag{1}$$

$A$ and $B$ represent the word vectors and $A_i$ or $B_i$ represent index i of that vector.
& Note that if A and B are identical, you will get $cos(\theta) = 1$.
* Otherwise, if they are the total opposite, meaning, $A= -B$, then you would get $cos(\theta) = -1$.
* If you get $cos(\theta) =0$, that means that they are orthogonal (or perpendicular).
* Numbers between 0 and 1 indicate a similarity score.
* Numbers between -1-0 indicate a dissimilarity score.

**Instructions**: Implement a function that takes in two word vectors and computes the cosine distance.

In [40]:
def cos_sim(A, B):
    dot = np.dot(A, B)
    norma = np.linalg.norm(A)
    normb = np.linalg.norm(B)
    cos = dot/(norma * normb)
    
    return cos

You will now implement a function that computes the similarity between two vectors using the Euclidean distance.
Euclidean distance is defined as:

$$ \begin{aligned} d(\mathbf{A}, \mathbf{B})=d(\mathbf{B}, \mathbf{A}) &=\sqrt{\left(A_{1}-B_{1}\right)^{2}+\left(A_{2}-B_{2}\right)^{2}+\cdots+\left(A_{n}-B_{n}\right)^{2}} \\ &=\sqrt{\sum_{i=1}^{n}\left(A_{i}-B_{i}\right)^{2}} \end{aligned}$$

* $n$ is the number of elements in the vector
* $A$ and $B$ are the corresponding word vectors. 
* The more similar the words, the more likely the Euclidean distance will be close to 0. 

**Instructions**: Write a function that computes the Euclidean distance between two vectors.

In [41]:
def euclidean(A, B):
    d = np.linalg.norm(A-B)
    return d

Usando os mesmos exemplos que fizemos com a medida de Jaccard, temos o seguinte:

In [42]:
data_dict[2][3], data_dict[3][3]

('DFA-', 'DFA-\n12')

In [43]:
cos_sim(get_vector(data_dict[2][3]), get_vector(data_dict[3][3]))

0.9486095485313412

In [44]:
euclidean(get_vector(data_dict[2][3]), get_vector(data_dict[3][3]))

1.9041294456687428

## Mapeando métricas de similaridade

Agora que já temos alguns procedimentos e estruturas auxiliares para medir similaridade entre os textos de anotações, vamos lá! A primeira coisa a se destacar é que temos alguns textos de anotações que foram anotados mais de 2 vezes, como mostra o mapa que fizemos mais cedo:

Elementos linha a linha parecem ter alguma relação (n -> n+1)

In [45]:
map_duplicata

{304: [15160],
 305: [1494],
 312: [9320, 9443, 9566, 9677],
 314: [15161],
 315: [1500],
 323: [15162],
 338: [9321, 9444, 9567, 9678],
 356: [1495],
 359: [12575],
 363: [9322, 9445, 9568, 9679, 15163],
 364: [9323, 9446, 9569, 9680],
 369: [12576],
 378: [12577],
 407: [9324, 9447, 9570, 9681],
 414: [15164],
 424: [15165],
 430: [1496],
 435: [9325, 9448, 9571, 9682],
 439: [12578],
 440: [1497],
 443: [15166],
 445: [9326, 9449, 9572, 9683],
 446: [9327, 9450, 9573, 9684],
 449: [12579],
 451: [1498],
 465: [12580],
 466: [1499],
 468: [9328, 9451, 9574, 9685],
 502: [1446],
 508: [1447],
 522: [9329, 9452, 9575, 9686],
 556: [9389, 9512, 9746, 9776],
 566: [1448],
 577: [9390, 9513, 9747, 9777, 15167],
 588: [9391, 9514, 9748, 9778],
 594: [1449],
 612: [15152],
 622: [15153],
 632: [9392, 9515, 9749, 9779],
 641: [9393, 9516, 9750, 9780],
 663: [15154],
 664: [15155],
 671: [9394, 9517, 9751, 9781],
 681: [1450],
 692: [9395, 9518, 9752, 9782],
 694: [12581],
 703: [15156],
 715

Nesse intuito, a estratégia adotada aqui foi criar combinações para cada uma das anotações feitas mais de uma vez. Para tanto, usamos o `itertools.combinations`. A ideia é calcular médias das combinações para cada métrica e salvar isso numa lista. Quando o elemento da vez for único, incluimos na lista 'el_unico'.

In [46]:
import itertools

In [47]:
cos_sim_lista = []
eucl_dist_lista = []
jacc_sim_lista = []

for offset in tqdm(offsets):
    if len(map_duplicata[offset]) > 1:
        qtd_comb = 0
        soma_cos = 0
        soma_eucl = 0
        soma_jc_char = 0
        # quero combinações dois a dois dos valores indexados por map_duplicata
        for combinacao in itertools.combinations(map_duplicata[offset], 2):
            #vector_1 = get_vector(data_dict[combinacao[0]][3])
            vector_1 = map_texto_vetores[combinacao[0]]
            #vector_2 = get_vector(data_dict[combinacao[1]][3])
            vector_2 = map_texto_vetores[combinacao[1]]
            cos = cos_sim(vector_1, vector_2)
            eucl = euclidean(vector_1, vector_2)
            jc = jaccard_char(data_dict[combinacao[0]][3], data_dict[combinacao[1]][3])
            soma_cos += cos
            soma_eucl += eucl
            soma_jc_char += jc
            qtd_comb += 1
        media_cos = soma_cos/qtd_comb
        media_eucl = soma_eucl/qtd_comb
        media_jc = soma_jc_char/qtd_comb
        cos_sim_lista.append(media_cos)
        eucl_dist_lista.append(media_eucl)
        jacc_sim_lista.append(media_jc)
    else:
        cos_sim_lista.append('el_unico')
        eucl_dist_lista.append('el_unico')
        jacc_sim_lista.append('el_unico')

  0%|          | 0/19492 [00:00<?, ?it/s]

Podemos ver que as listas possuem o mesmo número de linhas que o dataframe original.

In [48]:
len(cos_sim_lista), len(eucl_dist_lista), len(jacc_sim_lista)

(19492, 19492, 19492)

Para ficar mais bonitinho, vamos incluir isso no próprio dataframe:

In [49]:
df['sim_cos'] = cos_sim_lista
df['dist_eucl'] = eucl_dist_lista
df['sim_jacc'] = jacc_sim_lista

`sim_cos`, `dist_eucl`, `sim_jacc` foram as colunas adicionadas. 
- Para `sim_cos`, quanto mais próximo de 1, mais parecido. Se for = 1, é a mesma string;
- Para `dist_eucl`, quanto mais próximo de 0 melhor. Estamos falando de distância, então se a distância é 0, do ponto de vista vetorial é o mesmo elemento;
- Para `sim_jacc`, quanto mais próximo de 1, mais parecido. Se for = 1, é a mesma string.

In [50]:
df

,id_dodf,tipo_rel,id_rel,anotador_rel,tipo_ent,id_ent,anotador_ent,offset,length,texto,index,sim_cos,dist_eucl,sim_jacc
0,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,Ato_Exoneracao_Comissionado,1178,pedro_henrique,205008,326,EXONERAR SEBASTIAO FRANCISCO DE QUEIROZ do Car...,0,1.0,0.0,1.0
1,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,nome,1179,DODFMiner,205016,31,SEBASTIAO FRANCISCO DE QUEIROZ,1,1.0,0.0,1.0
2,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,simbolo,1181,DODFMiner,205078,4,DFA-,2,0.970634,1.088074,0.755102
3,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,simbolo,1180,pedro_henrique,205078,7,DFA-\n12,3,0.970634,1.088074,0.755102
4,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,cargo_comissionado,1182,DODFMiner,205089,9,Assessor,4,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19487,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,matricula_substituido,8685,vinicius_borges,644324,9,174.833-5,19487,el_unico,el_unico,el_unico
19488,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,data_inicial,8686,vinicius_borges,644348,2,09,19488,el_unico,el_unico,el_unico
19489,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,data_final,8687,vinicius_borges,644353,10,11/10/2013,19489,el_unico,el_unico,el_unico
19490,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,motivo,8688,vinicius_borges,644380,32,ferias regulamentares da titular,19490,el_unico,el_unico,el_unico


### ToDo:
- Polir o que já foi feito:
- Verificar questão do mapeamento de duplicatas: quando uma anotação é feita errada, o offset pode ser diferente
    - ou talvez pensar em uma posição relativa: pegar o offset do ato inteiro e ir caminhando length a length para encontrar as entidades anotadas; fazer assert com o texto da entidade em si. seria uma possibilidade para pegar casos como divergências de órgão e hierarquia de lotação, onde alguns anotadores anotam órgão pegando uma parte da hierarquia de lotação e vice-versa. é uma possibilidade para não depender exclusivamente do offset.
- Mapear médias por tipo de entidade;
    - basicamente é só uma consulta 'quase' sql via pandas com uma lambda function da vida. não fiz ainda pq minha barra de vida acabou :B
- Pensar nos critérios de concordância por label (kappa, floyd, krispendorf)